# Part 2 - Data Preprocessing and Modeling

This notebook will focus on preprocessing data to bring them optimal size and format using data augmentation parameters. Once the preprocessing complete, I will train various Tensorflow Keras "sequential" and ""convolutional" networks as well as pre-trained models to sample transfer learning to come up with best results. As the classes are imbalanced, my success metrics should be Precision and Recall. Specifically Recall score is the most important as our goal is to focus on minimizing false negative rates to not classify a patient as healthy while in fact they have pneumonia.

In [1]:
# Import libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from glob import glob

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, ZeroPadding2D, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For reproducibility
np.random.seed(42)

In [2]:
train_dir = './data/chest_xray/train/'
test_dir = './data/chest_xray/test/'

In [3]:
classes = ['normal', 'pneumonia']

I manually checked the images and found that there are a lot of variations for such a small dataset. The hight/width ratio, zooming range, angle of the body etc features differ among differen Xray images. Even the physical dimensions of images are vastly different. This makes it harder to train a model that will give high accuracy rate. I decided to use generator class to generate more images within train data with optimal rotation_range, shear_range, zoom_range, horizontal_flip (mirroring randomly selected images) to get additional observations to train the model with.

In [4]:
# Augmentation configuration to be used while training
train_generator = ImageDataGenerator(
                            rotation_range=20,
                            width_shift_range=0.25,
                            height_shift_range=0.25,
                            rescale=1./255,
                            shear_range=0.25,
                            zoom_range=0.25,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )

I will resize images to 224x224 px value and turn them to grayscale to only save the brightness and get rid of RGB values as the images are alrady provided as grayscale. It will help the train process run faster.


In [5]:
train_set = DirectoryIterator(train_dir,
                             train_generator,
                             target_size = (224, 224),
                             color_mode = 'rgb',
                             batch_size = 16,
                             classes=classes,
                             class_mode = 'categorical')

Found 5232 images belonging to 2 classes.


In [6]:
# Augmentation configuration to be used for validation
test_generator = ImageDataGenerator(rescale=1./255)

In [7]:
test_set = DirectoryIterator(test_dir,
                             test_generator,
                             target_size = (224, 224),
                             color_mode = 'rgb',
                             batch_size = 16, # set batch size a number that divides sample size
                             classes=classes,
                             class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [8]:
# Tensor size of train images
train_set.image_shape

(224, 224, 3)

In [9]:
# Tensor size of test images
test_set.image_shape

(224, 224, 3)

In [10]:
train_size = len(train_set.filenames)
test_size = len(test_set.filenames)

In [11]:
train_size, test_size

(5232, 624)

## Model #1 Dense

In [ ]:
# Initialize the sequential model
model1 = Sequential()

# Add flatten layer as input layer
model1.add(Flatten(input_shape = train_set.image_shape))

# Add a densely-connected layer with X neurons
model1.add(Dense(units = 10000,
                 activation='relu'))

# Add regularization
model1.add(Dropout(rate = 0.30))

# Add a second densely-connected layer with X neurons
model1.add(Dense(units = 128,
                activation='relu'))

# Add a third densely-connected layer with X neurons
model1.add(Dense(units = 64,
                 activation='relu'))

# Add regularization
model1.add(Dropout(rate = 0.10))

# Add a fourth densely-connected layer with X neurons
model1.add(Dense(units = 32,
                 activation='relu'))

# Add output layer
model1.add(Dense(units = 2,
    activation='sigmoid'
))

In [ ]:
# Define optimizer
opt = SGD(lr=0.01)

In [ ]:
# Compile the first model
model1.compile(optimizer = opt,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Define early stopping
early_stopping_monitor = EarlyStopping(patience = 5, 
                                       monitor = "val_accuracy", 
                                       mode="max", 
                                       verbose = 2)

# Define batch size (a divisor of test sample size)
batch_size = 16

In [ ]:
# Fit model on training data
history = model1.fit_generator(generator = train_set,
                              validation_data = test_set,
                              epochs = 25,
#                               callbacks=[early_stopping_monitor],
                              steps_per_epoch = train_size/batch_size, 
                              validation_steps = test_size/batch_size,
                              shuffle=False,
                              verbose = 2)

In [ ]:
predictions = model1.predict_generator(generator = test_set, 
                                       verbose = 2,
                                       steps = test_size/batch_size
                                      )

In [ ]:
y_hat = np.argmax(predictions, axis = 1) 
y_true = test_set.classes

In [ ]:
len(y_true) == len(y_hat)

In [ ]:
y_hat

In [ ]:
y_true

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
CM = confusion_matrix(y_true, y_hat)
fig, ax = plot_confusion_matrix(conf_mat=CM,  figsize=(5, 5))
plt.show()

## Model #2 CNN

In [ ]:
# Initialize the sequential model
model2 = Sequential()

# Add convolution and pooling as input layer
model2.add(Conv2D(filters = 32, # number of filters
                 kernel_size = (3, 3), # height/width of filter
                 input_shape = train_set.image_shape, # shape of input (image)
                 activation = 'relu')) # activation function

model2.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

# Add a second convolutional layer
model2.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3),
                 activation = 'relu'))

model2.add(MaxPooling2D(pool_size = (2, 2)))

# Add a flattening layer
model2.add(Flatten())

# Add a densely-connected layer
model2.add(Dense(units = 128,
                activation = 'relu'))

# Add output layer
model2.add(Dense(units = 2,
                activation = 'sigmoid'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Fit model on training data
history = model2.fit_generator(generator = train_set,
                              validation_data = test_set,
                              epochs = 10,
                              callbacks=[early_stopping_monitor],
                              steps_per_epoch = train_size/batch_size, 
                              validation_steps = test_size/batch_size,
                              shuffle = True,
                              verbose = 2)

## Model #3 - Pre-trained Model (VGG16 Convolutional Base)

I decided to use the artichecture of a pre-trained model as I was curious about transfer learning outcome. Transfer learning is using a pre-trained model and its weights on a different dataset. I chose to create an instance from VGG16 convolutional neural network model which is popular from ImageNet competition. This would allow me to save a lot of time while testing the performance of my data with an additional model.

In [12]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

Using TensorFlow backend.


In [13]:
# base_model = VGG16(weights = 'imagenet', 
#                  include_top = False,
#                  input_shape = train_set.image_shape)

In [14]:
model = Sequential()
model.add(Conv2D(input_shape=train_set.image_shape, filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="sigmoid"))

In [15]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [17]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='accuracy', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='max', save_freq=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [18]:
history = model.fit_generator(steps_per_epoch=100,
                              generator=train_set, 
                              validation_data=test_set, 
                              validation_steps=10,
                              epochs=100,
                              callbacks=[checkpoint,early])

Epoch 1/100

Epoch 00001: accuracy improved from -inf to 0.31250, saving model to vgg16_1.h5
  1/100 [..............................] - ETA: 1:10:19 - loss: 0.6933 - accuracy: 0.3125
Epoch 00001: accuracy improved from 0.31250 to 0.43750, saving model to vgg16_1.h5
  2/100 [..............................] - ETA: 1:02:53 - loss: 0.6994 - accuracy: 0.4375
Epoch 00001: accuracy improved from 0.43750 to 0.54167, saving model to vgg16_1.h5
  3/100 [..............................] - ETA: 59:29 - loss: 0.6969 - accuracy: 0.5417  
Epoch 00001: accuracy improved from 0.54167 to 0.54688, saving model to vgg16_1.h5
  4/100 [>.............................] - ETA: 55:52 - loss: 0.6954 - accuracy: 0.5469
Epoch 00001: accuracy improved from 0.54688 to 0.58750, saving model to vgg16_1.h5
  5/100 [>.............................] - ETA: 54:22 - loss: 0.6865 - accuracy: 0.5875
Epoch 00001: accuracy improved from 0.58750 to 0.59375, saving model to vgg16_1.h5
  6/100 [>.............................] - ETA

 52/100 [==============>...............] - ETA: 18:23 - loss: 0.6504 - accuracy: 0.7236
Epoch 00001: accuracy did not improve from 0.73234
 53/100 [==============>...............] - ETA: 17:56 - loss: 0.6516 - accuracy: 0.7205
Epoch 00001: accuracy did not improve from 0.73234
 54/100 [===============>..............] - ETA: 17:29 - loss: 0.6514 - accuracy: 0.7199
Epoch 00001: accuracy did not improve from 0.73234
 55/100 [===============>..............] - ETA: 17:03 - loss: 0.6508 - accuracy: 0.7193
Epoch 00001: accuracy did not improve from 0.73234
 56/100 [===============>..............] - ETA: 16:37 - loss: 0.6496 - accuracy: 0.7199
Epoch 00001: accuracy did not improve from 0.73234
 57/100 [================>.............] - ETA: 16:12 - loss: 0.6506 - accuracy: 0.7171
Epoch 00001: accuracy did not improve from 0.73234
 58/100 [================>.............] - ETA: 15:46 - loss: 0.6516 - accuracy: 0.7144
Epoch 00001: accuracy did not improve from 0.73234
 59/100 [================>.

  9/100 [=>............................] - ETA: 29:49 - loss: 0.5523 - accuracy: 0.7708
Epoch 00002: accuracy did not improve from 0.87500
 10/100 [==>...........................] - ETA: 29:22 - loss: 0.5536 - accuracy: 0.7688
Epoch 00002: accuracy did not improve from 0.87500
 11/100 [==>...........................] - ETA: 28:56 - loss: 0.5545 - accuracy: 0.7670
Epoch 00002: accuracy did not improve from 0.87500
 12/100 [==>...........................] - ETA: 28:32 - loss: 0.5445 - accuracy: 0.7760
Epoch 00002: accuracy did not improve from 0.87500
 13/100 [==>...........................] - ETA: 28:08 - loss: 0.5352 - accuracy: 0.7837
Epoch 00002: accuracy did not improve from 0.87500
 14/100 [===>..........................] - ETA: 27:46 - loss: 0.5536 - accuracy: 0.7679
Epoch 00002: accuracy did not improve from 0.87500
 15/100 [===>..........................] - ETA: 27:22 - loss: 0.5699 - accuracy: 0.7542
Epoch 00002: accuracy did not improve from 0.87500
 16/100 [===>..............

 68/100 [===================>..........] - ETA: 10:07 - loss: 0.5590 - accuracy: 0.7583
Epoch 00002: accuracy did not improve from 0.87500
 69/100 [===================>..........] - ETA: 9:48 - loss: 0.5593 - accuracy: 0.7582 
Epoch 00002: accuracy did not improve from 0.87500
 70/100 [====================>.........] - ETA: 9:29 - loss: 0.5594 - accuracy: 0.7580
Epoch 00002: accuracy did not improve from 0.87500
 71/100 [====================>.........] - ETA: 9:10 - loss: 0.5595 - accuracy: 0.7579
Epoch 00002: accuracy did not improve from 0.87500
 72/100 [====================>.........] - ETA: 8:51 - loss: 0.5605 - accuracy: 0.7569
Epoch 00002: accuracy did not improve from 0.87500
 73/100 [====================>.........] - ETA: 8:32 - loss: 0.5587 - accuracy: 0.7586
Epoch 00002: accuracy did not improve from 0.87500
 74/100 [=====================>........] - ETA: 8:13 - loss: 0.5579 - accuracy: 0.7593
Epoch 00002: accuracy did not improve from 0.87500
 75/100 [=====================>.

 26/100 [======>.......................] - ETA: 23:19 - loss: 0.5841 - accuracy: 0.7332
Epoch 00003: accuracy did not improve from 0.87500
 27/100 [=======>......................] - ETA: 23:00 - loss: 0.5833 - accuracy: 0.7338
Epoch 00003: accuracy did not improve from 0.87500
 28/100 [=======>......................] - ETA: 22:41 - loss: 0.5804 - accuracy: 0.7366
Epoch 00003: accuracy did not improve from 0.87500
 29/100 [=======>......................] - ETA: 22:23 - loss: 0.5903 - accuracy: 0.7263
Epoch 00003: accuracy did not improve from 0.87500
 30/100 [========>.....................] - ETA: 22:04 - loss: 0.5895 - accuracy: 0.7271
Epoch 00003: accuracy did not improve from 0.87500
 31/100 [========>.....................] - ETA: 21:45 - loss: 0.5906 - accuracy: 0.7258
Epoch 00003: accuracy did not improve from 0.87500
 32/100 [========>.....................] - ETA: 21:26 - loss: 0.5898 - accuracy: 0.7266
Epoch 00003: accuracy did not improve from 0.87500
 33/100 [========>.........

...

 17/100 [====>.........................] - ETA: 18:37 - loss: 0.5665 - accuracy: 0.7500
Epoch 00006: accuracy did not improve from 0.90625
 18/100 [====>.........................] - ETA: 18:31 - loss: 0.5664 - accuracy: 0.7500
Epoch 00006: accuracy did not improve from 0.90625
 19/100 [====>.........................] - ETA: 18:19 - loss: 0.5569 - accuracy: 0.7599
Epoch 00006: accuracy did not improve from 0.90625
 20/100 [=====>........................] - ETA: 18:12 - loss: 0.5602 - accuracy: 0.7563
Epoch 00006: accuracy did not improve from 0.90625
 21/100 [=====>........................] - ETA: 18:03 - loss: 0.5604 - accuracy: 0.7560
Epoch 00006: accuracy did not improve from 0.90625
 22/100 [=====>........................] - ETA: 17:55 - loss: 0.5549 - accuracy: 0.7614
Epoch 00006: accuracy did not improve from 0.90625
 23/100 [=====>........................] - ETA: 17:44 - loss: 0.5499 - accuracy: 0.7663
Epoch 00006: accuracy did not improve from 0.90625
 24/100 [======>...........

 76/100 [=====================>........] - ETA: 5:57 - loss: 0.5582 - accuracy: 0.7566
Epoch 00006: accuracy did not improve from 0.90625
 77/100 [======================>.......] - ETA: 5:43 - loss: 0.5608 - accuracy: 0.7541
Epoch 00006: accuracy did not improve from 0.90625
 78/100 [======================>.......] - ETA: 5:28 - loss: 0.5617 - accuracy: 0.7532
Epoch 00006: accuracy did not improve from 0.90625
 79/100 [======================>.......] - ETA: 5:13 - loss: 0.5591 - accuracy: 0.7555
Epoch 00006: accuracy did not improve from 0.90625
 80/100 [=======================>......] - ETA: 4:58 - loss: 0.5609 - accuracy: 0.7539
Epoch 00006: accuracy did not improve from 0.90625
 81/100 [=======================>......] - ETA: 4:43 - loss: 0.5617 - accuracy: 0.7531
Epoch 00006: accuracy did not improve from 0.90625
 82/100 [=======================>......] - ETA: 4:28 - loss: 0.5617 - accuracy: 0.7530
Epoch 00006: accuracy did not improve from 0.90625
 83/100 [=======================>.

 34/100 [=========>....................] - ETA: 16:00 - loss: 0.5795 - accuracy: 0.7353
Epoch 00007: accuracy did not improve from 0.90625
 35/100 [=========>....................] - ETA: 15:46 - loss: 0.5772 - accuracy: 0.7375
Epoch 00007: accuracy did not improve from 0.90625
 36/100 [=========>....................] - ETA: 15:34 - loss: 0.5786 - accuracy: 0.7361
Epoch 00007: accuracy did not improve from 0.90625
 37/100 [==========>...................] - ETA: 15:20 - loss: 0.5798 - accuracy: 0.7348
Epoch 00007: accuracy did not improve from 0.90625
 38/100 [==========>...................] - ETA: 15:07 - loss: 0.5827 - accuracy: 0.7319
Epoch 00007: accuracy did not improve from 0.90625
 39/100 [==========>...................] - ETA: 14:51 - loss: 0.5871 - accuracy: 0.7276
Epoch 00007: accuracy did not improve from 0.90625
 40/100 [===========>..................] - ETA: 14:38 - loss: 0.5881 - accuracy: 0.7266
Epoch 00007: accuracy did not improve from 0.90625
 41/100 [===========>......

 93/100 [==========================>...] - ETA: 1:45 - loss: 0.5742 - accuracy: 0.7399
Epoch 00007: accuracy did not improve from 0.90625
 94/100 [===========================>..] - ETA: 1:30 - loss: 0.5757 - accuracy: 0.7387
Epoch 00007: accuracy did not improve from 0.90625
 95/100 [===========================>..] - ETA: 1:15 - loss: 0.5740 - accuracy: 0.7401
Epoch 00007: accuracy did not improve from 0.90625
 96/100 [===========================>..] - ETA: 1:00 - loss: 0.5739 - accuracy: 0.7402
Epoch 00007: accuracy did not improve from 0.90625
 97/100 [============================>.] - ETA: 45s - loss: 0.5762 - accuracy: 0.7384 
Epoch 00007: accuracy did not improve from 0.90625
 98/100 [============================>.] - ETA: 30s - loss: 0.5745 - accuracy: 0.7398
Epoch 00007: accuracy did not improve from 0.90625
 99/100 [============================>.] - ETA: 15s - loss: 0.5752 - accuracy: 0.7393
Epoch 00007: accuracy did not improve from 0.90625
100/100 [===========================

FileNotFoundError: [Errno 2] No such file or directory: './data/chest_xray/train/pneumonia\\person1261_virus_2147.jpeg'